In [1]:
# import library for Support Vector Classification
from sklearn.svm import SVC

# import library for plot
import matplotlib.pyplot as plt

# import library for data handling
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import math

# import library for pytorch
import torch
import torch.nn as nn

# import custom function
from stock_predict import SequenceDataset, ShallowRegressionLSTM, train, test, get_stock_data, split_train_test, predict

# Get cuda device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# predict df
predict_df = pd.DataFrame(columns = ["High", "Low", "Open", "Close"])

/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get the past stock price
df = get_stock_data(stock_name = 'TSLA', source = 'yahoo', past_year = 2)


In [3]:
pred_list = ["High", "Low", "Open", "Close"]

In [9]:
def run(df,training_target):

    # Split data into train and test
    train_df, test_df, feature, target, train_size, test_size, target_mean, target_std = split_train_test(training_target = training_target, df = df, train_ratio = 0.8)

    # create dataset and data loader
    batch_size = 4
    seq_len = 60

    torch.manual_seed(seq_len*batch_size/2)

    train_dataset = SequenceDataset(train_df, target, feature, seq_len)
    test_dataset = SequenceDataset(test_df, target, feature, seq_len)
    train_looader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # get the first batch and its shape
    x, y = next(iter(train_looader))
    print("Features shape: ", x.shape)
    print("Target shape: ", y.shape)

    # create model, loss and optimizer
    learning_rate = 5e-5
    num_hidden = 16
    epochs = 40

    model = ShallowRegressionLSTM(n_features = len(feature), n_hidden = num_hidden, device=device).to(device)
    loss = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # train and test model
    print("Untrained test\n----------------")
    test(model, test_loader, loss, device)
    print("\nTraining\n----------------")

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}\n")
        train(model, train_looader, loss, optimizer, device)
        test(model, test_loader, loss, device)

    train_eval = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    prediction = f"{target}_pred"
    train_df[prediction] = predict(train_eval, model, device)
    test_df[prediction] = predict(test_loader, model, device)

    df_out = pd.concat([train_df, test_df], axis=0)[[target, prediction]]

    for c in df_out.columns:
        df_out[c] = df_out[c] * target_std + target_mean
    
    return df_out


In [10]:
result_list = []
for training_target in pred_list:
    print(f"Training target: {training_target}")
    df_out = run(df,training_target)
    result_list.append(df_out)

Training target: High
Features shape:  torch.Size([4, 60, 6])
Target shape:  torch.Size([4])
Untrained test
----------------
Test loss: 0.2787

Training
----------------
Epoch 1

Train loss: 1.0407
Test loss: 0.2585
Epoch 2

Train loss: 0.9764
Test loss: 0.2396
Epoch 3

Train loss: 0.9114
Test loss: 0.2207
Epoch 4

Train loss: 0.8493
Test loss: 0.2015
Epoch 5

Train loss: 0.7864
Test loss: 0.1817
Epoch 6

Train loss: 0.7228
Test loss: 0.1613
Epoch 7

Train loss: 0.6581
Test loss: 0.1401
Epoch 8

Train loss: 0.5945
Test loss: 0.1176
Epoch 9

Train loss: 0.5258
Test loss: 0.0951
Epoch 10

Train loss: 0.4641
Test loss: 0.0757
Epoch 11

Train loss: 0.4048
Test loss: 0.0589
Epoch 12

Train loss: 0.3492
Test loss: 0.0468
Epoch 13

Train loss: 0.3039
Test loss: 0.0402
Epoch 14

Train loss: 0.2638
Test loss: 0.0383
Epoch 15

Train loss: 0.2333
Test loss: 0.0402
Epoch 16

Train loss: 0.2074
Test loss: 0.0442
Epoch 17

Train loss: 0.1873
Test loss: 0.0488
Epoch 18

Train loss: 0.1718
Test loss: 

In [14]:
result_list[3]

,Close_t+1,Close_t+1_pred
Date,,
2020-09-15,147.253326,157.035095
2020-09-16,141.143326,155.430847
2020-09-17,147.383331,153.778137
2020-09-18,149.796661,153.841644
2020-09-21,141.410004,154.715057
...,...,...
2022-09-08,299.679993,284.211426
2022-09-09,304.420013,290.911285
2022-09-12,292.130005,295.391266
